### Teste -Web Scraping e Crawlers

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
import concurrent.futures
import json


In [7]:
url = "http://www.freeproxylists.net"
requisicao = requests.get(url)
site = BeautifulSoup(requisicao.text, "html.parser") #le o html 
print(site.prettify())

<html>
 <head>
  <title>
   Free Proxy Lists
  </title>
  <style type="text/css">
   <!--
div#contents {
width: 350px;
height: 200px;
position: absolute;
top: 50%;
left: 50%;
margin: -100px 0 0 -175px;
padding: 0px;
}
-->
  </style>
  <script async="" defer="" src="https://www.google.com/recaptcha/api.js">
  </script>
 </head>
 <body>
  <div id="contents">
   <center>
    <form action="/" method="post">
     <div class="g-recaptcha" data-sitekey="6LdOqaobAAAAACFOBruA5lJuP0FEKN4vmdz4BMRV">
     </div>
     <br/>
     <input type="submit" value="Submit"/>
    </form>
   </center>
  </div>
 </body>
</html>



In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor


In [5]:
#função para extrair da tabela os elementos (células)
def extrair_dados(linha):
    # Obtenha os elementos da linha (td)
    elementos = linha.find_all("td")
    
    if len(elementos) < 9:
        return None  #Ignore linhas com elementos insuficientes e passe para a próxima
    
    try:
        ip = elementos[0].find("a").text.strip()
        porta = elementos[1].text.strip()
        protocolo = elementos[2].text.strip()
        país = elementos[4].text.strip()
        uptime = elementos[7].text.strip()
    except Exception as e:
        print(f"Erro ao extrair dados da linha: {e}")
        return None
    
    proxy = {
        "ip": ip,
        "porta": porta,
        "protocolo": protocolo,
        "pais": país,
        "uptime": uptime
    }
    
    return proxy


In [22]:

#desabilitando funcionalidades relacionadas ao controle automatizado. 
# Essa exclusão indica ao navegador que não está sendo usado uma automação para controlá-lo, 
# o que pode ajudar a evitar a detecção por parte dos sites que possam bloquear o acesso automatizado.
 
options = webdriver.ChromeOptions() 
# desabilitar a AutomationControlled flag 
options.add_argument("--disable-blink-features=AutomationControlled") 
# Exclui a coleção de opções de habilitação de automação
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 

options.add_experimental_option("useAutomationExtension", False) # Desliga userAutomationExtension 
driver = webdriver.Chrome(options=options) 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
driver.get("http://www.freeproxylists.net")  

# Verifica se o CAPTCHA está presente na página.
if len(driver.find_elements(By.CLASS_NAME, "g-recaptcha")) > 0:
    print("Por favor, resolva o CAPTCHA manualmente no navegador.")
    input("Pressione Enter após resolver o CAPTCHA no navegador...")

else:
    print("Não há CAPTCHA - Continuando com a raspagem de dados")

start_time = time.time() #tempo inicial - inicio da raspagem

# Número total de páginas a percorrer
total_pages = 7

start_time = time.time() # inicio da raspagem

# Cria uma lista vazia para armazenar todos os proxies
all_proxies = []

for page in range(1, total_pages + 1):
    # Construçao da URL da página 
    url = f"http://www.freeproxylists.net/?page={page}"

    driver.get(url)

    # Retorna o código-fonte HTML da página web atualmente carregada no navegador controlado pelo WebDriver.
    html = driver.page_source

    # Cria um objeto BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(html, "html.parser")

    # Encontra a tabela que contém os dados
    tabela = soup.find("table", class_="DataGrid")

    # Verifica se a tabela foi encontrada
    if tabela is None:
        print(f"Tabela não encontrada na página {page}. Pulando para a próxima página.")
        continue

    # Encontra todas as linhas da tabela que possuem as classes 'Odd' ou 'Even'
    linhas = tabela.select("tr.Odd, tr.Even")

    # Define o número máximo de threads que deseja utilizar
    num_threads = 10
    proxies = []
    # Cria um ThreadPoolExecutor com o número de threads desejado
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        resultados = executor.map(extrair_dados, linhas)

        # Itera pelos resultados
        for resultado in resultados:
            if resultado is not None:
                 proxies.append(resultado)

    print(f"Extraídos dados da página {page}.")
    for proxy in proxies:
        print(proxy)
    # Adiciona os proxies da página atual à lista all_proxies
    all_proxies.extend(proxies)


# Verifica se a lista all_proxies contém algum proxy
if len(all_proxies) > 0:
    with open('proxies.json', 'w') as file:
        json.dump(all_proxies, file, indent=4)
    print('JSON salvo com sucesso!')
else:
    print('Nenhum proxy encontrado para salvar no JSON.')

end_time = time.time() #tempo final
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")
driver.quit()


Não há CAPTCHA - Continuando com a raspagem de dados
Extraídos dados da página 1.
{'ip': '103.148.25.223', 'porta': '8080', 'protocolo': 'HTTP', 'pais': 'Indonesia', 'uptime': '2.4%'}
{'ip': '45.175.237.161', 'porta': '999', 'protocolo': 'HTTPS', 'pais': 'Mexico', 'uptime': '2.7%'}
{'ip': '179.61.229.62', 'porta': '999', 'protocolo': 'HTTP', 'pais': 'Dominican Republic', 'uptime': '12.5%'}
{'ip': '45.81.144.23', 'porta': '8080', 'protocolo': 'HTTP', 'pais': 'Iraq', 'uptime': '4.7%'}
{'ip': '47.243.50.83', 'porta': '80', 'protocolo': 'HTTPS', 'pais': 'Hong Kong', 'uptime': '0.7%'}
{'ip': '198.199.86.11', 'porta': '3128', 'protocolo': 'HTTP', 'pais': 'United States', 'uptime': '17.0%'}
{'ip': '190.104.20.85', 'porta': '8080', 'protocolo': 'HTTP', 'pais': 'Bolivia', 'uptime': '2.2%'}
{'ip': '89.175.26.210', 'porta': '80', 'protocolo': 'HTTP', 'pais': 'Russia', 'uptime': '3.5%'}
{'ip': '103.139.47.170', 'porta': '8080', 'protocolo': 'HTTP', 'pais': 'Indonesia', 'uptime': '2.8%'}
{'ip': '11

In [23]:
!pip freeze > requirements.txt